In [ ]:
#Ngoc Tuan HOANG - s147568
#Thesis topic: Heuristic solution approach for the Capacitated Lot-sizing and Scheduling Problem with 
#Sequence-Dependent Setup Times and Costs

#This code is used for the Heuristic 1 (Fix and Relax heuristic) presented in the thesis. 

In [1]:
using JuMP          
using Gurobi        

In [2]:
#In this box, three parameters need to be fulfilled. 
#inst: number of the instance being tested
#N: number of products
#T: number of time periods

#Information about instances can be found in the Appendix 2 at the end of the thesis.

inst = 8
Data = readdlm("Instance$(inst).txt")
N=5
T=20

20

In [3]:
#These lines of code are used to read all deterministic parameters (input data). 

d=cell(N,T)
for i=1:N
    for j=1:T
        d[i,j]=Data[i,j]
    end
end
r=cell(N,T)
for i=1:N
    for j=1:T
        r[i,j]=Data[N+i,j]
    end
end
hn=cell(N,T)
for i=1:N
    for j=1:T
        hn[i,j]=Data[2*N+i,j]
    end
end
hr=cell(N,T)
for i=1:N
    for j=1:T
        hr[i,j]=Data[3*N+i,j]
    end
end
an=cell(N,T)
for i=1:N
    for j=1:T
        an[i,j]=Data[4*N+i,j]
    end
end
ar=cell(N,T)
for i=1:N
    for j=1:T
        ar[i,j]=Data[5*N+i,j]
    end
end
p=cell(N,T)
for i=1:N
    for j=1:T
        p[i,j]=Data[6*N+i,j]
    end
end
time=cell(N,N,T)
for t=1:T
    time[:,:,t]=Data[7*N+((t-1)*N+1):7*N+((t-1)*N+N),1:N]
end
cost=cell(N,N,T)
for t=1:T
    cost[:,:,t]=Data[7*N+N*T+((t-1)*N+1):7*N+N*T+((t-1)*N+N),1:N]
end
cap=cell(1,T)
for j=1:T
    cap[1,j]=Data[7*N+2*N*T+1,j]
end
b=cell(N,1)
for i=1:N
    b[i,1]=Data[7*N+2*N*T+1+i,1]
end
jo=Data[8*N+2*N*T+2,1]

3

In [4]:
#The heuristic is organized in a function command. Function FR 's mission is to generate and solve the 
#sub-problem. For each value of L (from 1 to NO), one sub-problem will be formulated and solved. 
#In the code, some constraints will be active or inactive, depending on the value of L.

function FR(L)
    
#The model and variables are defined.
    
m=Model(solver=GurobiSolver())
@defVar(m,XN[i=1:N,t=1:T,s=1:N]>=0)
@defVar(m,XR[i=1:N,t=1:T,s=1:N]>=0)
@defVar(m,InvN[i=1:N,t=1:T]>=0)
@defVar(m,InvR[i=1:N,t=1:T]>=0)
@defVar(m,B[i=1:N,t=1:T]>=0)
@defVar(m,0<=Y[i=1:N,j=1:N,t=1:T,s=1:N]<=1)
@defVar(m,R[i=1:N,j=1:N,e=1:St,s=1:N],Bin)


#Objective function
    
@setObjective(m,Min,sum{an[i,t]*XN[i,t,s]+ar[i,t]*XR[i,t,s],i=1:N,t=1:T,s=1:N}
+sum{hn[i,t]*InvN[i,t]+hr[i,t]*InvR[i,t],i=1:N,t=1:T}
+sum{cost[i,j,t]*Y[i,j,t,s],i=1:N,j=1:N,t=1:T,s=1:N;j !=i}+sum{b[i]*B[i,t],i=1:N,t=1:T})

#(1.a) Inventory balance constraint needs to be splited into 2 constraints, one for t=1 and another for 
#t=2:N

for i=1:N
    for t=2:T             
        @addConstraint(m,InvN[i,t]-B[i,t]==InvN[i,t-1]-B[i,t-1]+sum{XN[i,t,s]+XR[i,t,s],s=1:N}-d[i,t])  
    end
end
for i=1:N
    for t=1           
        @addConstraint(m,InvN[i,t]-B[i,t]==sum{XN[i,t,s]+XR[i,t,s],s=1:N}-d[i,t])
    end
end
    
#(1.b) The same with the inventory balance of return products    

for i=1:N
    for t=2:T            
        @addConstraint(m,InvR[i,t]==InvR[i,t-1]+r[i,t]-sum{XR[i,t,s],s=1:N})
    end
end
for i=1:N
    for t=1            
        @addConstraint(m,InvR[i,t]==r[i,t]-sum{XR[i,t,s],s=1:N})
    end
end

#Constraint (2)    
    
for i=1:N       
    for t=T     
        @addConstraint(m,B[i,t]==0)
    end
end
    
#(3) Capacity constraint
    
for t=1:T       
        @addConstraint(m,sum{p[i,t]*(XN[i,t,s]+XR[i,t,s]),i=1:N,s=1:N}+
                sum{time[i,j,t]*Y[i,j,t,s],i=1:N,j=1:N,s=1:N}<=cap[t])
end

#Constraint (4)

for j=1:N
    for i=1:N
        if j!=jo
            @addConstraint(m,Y[j,i,1,1]==0)
        end
    end
end   
    
#Constraint (5)
    
@addConstraint(m,sum{Y[jo,i,1,1],i=1:N}==1)

#Constraint (6)

for i=1:N
    for t=1:T
        for s=1:(N-1)
             @addConstraint(m,sum{Y[j,i,t,s],j=1:N}==sum{Y[i,k,t,s+1],k=1:N})    
        end
    end
end
    
#Constraint (7)    
    
for i=1:N
    for t=2:T
        @addConstraint(m,sum{Y[j,i,t-1,N],j=1:N}==sum{Y[i,k,t,1],k=1:N})    
    end
end
    
#Constraint (8)    
    
for i=1:N
    for t=1:T
        for s=1:N
                @addConstraint(m,XN[i,t,s]+XR[i,t,s]<=sum(d)*sum{Y[j,i,t,s],j=1:N})    
        end
    end
end
    
#Constraint (9) 
    
for t=1:T
    for s=1:N
            @addConstraint(m,sum{Y[j,i,t,s],i=1:N,j=1:N}<=1)
    end
end
    
#Constraint (10)   
    
for j=1:N
    for t=1:T 
        @addConstraint(m,sum{Y[i,j,t,s],i=1:N,s=1:N}<=1)    
    end
end

#The first constraint presented in section 3.1. Heuristic 1
     
if (L-1)*F+St<=T
for i=1:N
    for j=1:N
         for e=1:St
            for s=1:N
                    @addConstraint(m,Y[i,j,(L-1)*F+e,s]==R[i,j,e,s])
            end
        end
    end
end
    else 
    for i=1:N
    for j=1:N
    for e=1:T-(L-1)*F
    for s=1:N
                @addConstraint(m,Y[i,j,(L-1)*F+e,s]==R[i,j,e,s])
    end
    end
    end
    end
end

#The second constraint presented in section 3.1. Heuristic 1    
    
if L>1 
for i=1:N
    for j=1:N
        for t=1:(L-1)*F
            for s=1:N
                @addConstraint(m,Y[i,j,t,s]==KK[i,j,t,s])
            end
        end
    end
end
end
    
#The sub-problem in this iteration is solved
solve(m)

#Optimal solution of the iteration will be recorded in matrices outside the function FR
    
if L==NO
XNs=getValue(XN)
 for i=1:N
        for t=1:T
            for s=1:N
                XNsol[i,t,s]=XNs[i,t,s]
            end
        end
    end
XRs=getValue(XR)
    for i=1:N
        for t=1:T
            for s=1:N
                XRsol[i,t,s]=XRs[i,t,s]
            end
        end
    end
Bs=getValue(B)
    for i=1:N
        for t=1:T
            Bsol[i,t]=Bs[i,t]
        end
    end
    end
Ysol =getValue(Y)
    for i=1:N
        for j=1:N
            for t=1:T
                for s=1:N
                    KK[i,j,t,s]=Ysol[i,j,t,s]
                end
            end
        end
    end
end

FR (generic function with 1 method)

In [5]:
#Before running function FR, following matrices need to be defined.

KK=zeros(N,N,T,N)
XNsol=cell(N,T,N)
XRsol=cell(N,T,N)
Bsol=cell(N,T)

#Parameter St and F are chosen depending on the experiment. NO is the number of iteration that the 
#heuristic will run, depdending on F. NO = T/F and round up to the closest integer.

St=1
F=1
NO=20

tic()
for L=1:NO
    FR(L)
    println(L)
end
toc()

#Best objective of the last iteration is also the objective value of the heuristic for the orginal 
#problem. 
#CPU time will also appear at the end of the result box.

Optimize a model with 1566 rows, 3925 columns and 18815 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+04]
  Objective range [1e+00, 6e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 2e+04]
Presolve removed 151 rows and 175 columns
Presolve time: 0.03s
Presolved: 1415 rows, 3750 columns, 18328 nonzeros
Variable types: 3665 continuous, 85 integer (85 binary)

Root relaxation: objective 3.227788e+05, 1349 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 322778.790    0   14          - 322778.790      -     -    0s
H    0     0                    322825.43081 322778.790  0.01%     -    0s
     0     0 322796.177    0   11 322825.431 322796.177  0.01%     -    0s

Cutting planes:
  Gomory: 3
  Implied bound: 2
  MIR: 6

Explored 0 nodes (1382 simplex iterations) in 0.10 seconds
Thread count was 4 (of 4 available processo

8.029666582

.029666582 seconds


In [6]:
#Matrix Bsol is the solution of decision variable B in the original problem.

sum(Bsol)

68.99999999999996

In [7]:
#Matrix XNsol is the solution of decision variable XN in the original problem.

XNsol

5x20x5 Array{Any,3}:
[:, :, 1] =
   0.0    298.0    0.0    22.0    0.0   …  0.0    0.0  504.0  0.0   0.0   
   0.0      0.0  267.567   0.0    0.0      0.0    0.0    0.0  0.0   0.0   
 519.656    0.0    0.0     0.0    0.0      0.0  566.0    0.0  0.0   0.0   
   0.0      0.0    0.0     0.0  137.31     0.0    0.0    0.0  0.0   0.0   
   0.0      0.0    0.0     0.0    0.0      0.0    0.0    0.0  0.0  45.5351

[:, :, 2] =
   0.0  0.0  0.0    0.0  0.0  0.0  …  370.913    0.0    0.0  0.0    0.0  
   0.0  0.0  0.0    0.0  0.0  0.0       0.0    255.261  0.0  0.0    0.0  
   0.0  0.0  0.0    0.0  0.0  0.0       0.0      0.0    0.0  0.0    0.0  
   0.0  0.0  0.0    0.0  0.0  0.0       0.0      0.0    0.0  0.0  101.525
 233.0  0.0  0.0  343.0  0.0  0.0       0.0      0.0    0.0  0.0    0.0  

[:, :, 3] =
  0.0    0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0    0.0
  0.0    0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  205.0
  0.0    0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0

In [8]:
#Matrix XRsol is the solution of decision variable XR in the original problem.

XRsol

5x20x5 Array{Any,3}:
[:, :, 1] =
 0.0  0.0   0.0  109.0   0.0  42.0  …   0.0      0.0  79.6854   0.0   0.0
 0.0  0.0  91.0    0.0   0.0   0.0     71.1929   0.0   0.0     87.0   0.0
 0.0  0.0   0.0    0.0   0.0   0.0      0.0     81.0   0.0      0.0   0.0
 0.0  0.0   0.0    0.0  40.0   0.0      0.0      0.0   0.0      0.0   0.0
 0.0  0.0   0.0    0.0   0.0   0.0      0.0      0.0   0.0      0.0  86.0

[:, :, 2] =
 0.0   0.0   0.0  0.0  0.0  0.0   0.0  …  135.087  0.0  0.0    0.0     0.0
 0.0   0.0   0.0  0.0  0.0  0.0   0.0       0.0    0.0  0.0    0.0     0.0
 0.0   0.0   0.0  0.0  0.0  0.0   0.0       0.0    0.0  0.0    0.0     0.0
 0.0   0.0   0.0  0.0  0.0  0.0   0.0       0.0    0.0  0.0  384.475  87.0
 0.0  66.0  34.0  0.0  0.0  0.0  95.0       0.0    0.0  0.0    0.0     0.0

[:, :, 3] =
  0.0  0.0    0.0  0.0   0.0  0.0  0.0  …  0.0  0.0   0.0  0.0   0.0   0.0
  0.0  0.0    0.0  0.0   0.0  0.0  0.0     0.0  0.0   0.0  0.0   0.0  85.0
  0.0  0.0  109.0  0.0   0.0  0.0  0.0     0.0

In [9]:
#With this box, we can compute the total capacity used by the solution in order to confirm the 
#valid of the solution.

Ysol=KK
capUsed=0
for i=1:N
    for s=1:N
        for t=1:T
            capUsed=capUsed+(XNsol[i,t,s]+XRsol[i,t,s])*p[i,t]
        end
    end
end
for i=1:N
    for j=1:N
        for t=1:T
            for s=1:N
                capUsed=capUsed+time[i,j,t]*Ysol[i,j,t,s]
            end
        end
    end
end
capUsed

368874.3363913406

In [10]:
#Total avaiable capacity, to compare with capUsed.

sum(cap)

374000